In [8]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "dirty_cafe_sales.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training",
  file_path
)

C:\Users\PC\AppData\Local\Temp\ipykernel_2008\1585058692.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


In [37]:
# Running this to get a glimpse of the data
df.head(10)

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5.0,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,Cake/Juice,3.0,3.0,9.0,NaN,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4.0,4.0,16.0,Cash,NaN,2023-10-28
8,TXN_4717867,Cake/Juice,5.0,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5.0,4.0,20.0,NaN,In-store,2023-12-31


In [10]:
# Standardize column names.
# Remove leading/trailing spaces, replace spaces with underscores, and convert to lowercase.
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()

In [ ]:
# First, we want to investigate data quality issues for each column.
# So, we check for each column the unique values and their counts including NaNs.
# Loop through every single column in the dataframe
# then we read the results to try catch anything unusual.
for col in df.columns:
    print(f" Column: {col}")
    
    # Show Value Counts including NaNs 
    print("Value Counts:")
    print(df[col].value_counts(dropna=False))
    # This is just separator for readibility
    print("-" * 30 + "\n")

 Column: transaction_id
Value Counts:
transaction_id
TXN_9226047    1
TXN_8567525    1
TXN_4583012    1
TXN_6796890    1
TXN_9933628    1
              ..
TXN_3160411    1
TXN_7034554    1
TXN_4271903    1
TXN_4977031    1
TXN_1961373    1
Name: count, Length: 10000, dtype: int64
------------------------------

 Column: item
Value Counts:
item
Coffee               1284
Salad                1270
Cookie               1209
Tea                  1199
Juice                1171
Cake                 1139
Sandwich             1131
Smoothie             1096
Cake/Juice            234
Smoothie/Sandwich     213
NaN                    54
Name: count, dtype: int64
------------------------------

 Column: quantity
Value Counts:
quantity
5      2013
2      1974
4      1863
3      1849
1      1822
NaN     479
Name: count, dtype: int64
------------------------------

 Column: price_per_unit
Value Counts:
price_per_unit
3.0    2429
4.0    2331
2.0    1227
5.0    1204
1.0    1143
1.5    1133
NaN     533
Na

In [12]:
# Based on the value counts above, we can see that there are several columns that have "ERROR", "UNKNOWN" and nan values.
# We will standardize these values to be nan for easier handling.

import numpy as np

# Define your mapping rules
rules = {
    'ERROR': np.nan,
    'UNKNOWN': np.nan
}

# Apply to the entire DataFrame
df = df.replace(rules)

FILLNA FOR ITEM COLUMN

In [5]:
# Now, if we take a look at all rows where item is "Juice", the value for price_per_unit is always 3.0.
df[df['item'] == "Juice"]['price_per_unit']

17      3.0
19      3.0
27      3.0
43      3.0
46      3.0
       ... 
9960    3.0
9967    3.0
9977    3.0
9987    3.0
9989    3.0
Name: price_per_unit, Length: 1171, dtype: object

In [ ]:
# Next,if we take a look at all rows where item is "Coffee", the value for price_per_unit is always 2.0.
# We can see there is consistency in value of price_per_unit for different items
# Meaning we can impute missing or unknown items based on the price per unit.
df[df['item'] == "Coffee"]['price_per_unit']

In [ ]:
# Create a mapping dictionary from price_per_unit to item
price_grouping = (
    df[df['item'].notna()]  #  filter out NaN values, we only want to map known items.
    .groupby('price_per_unit')['item'] # group by price_per_unit and get the item values. e.g price_per_unit = 2.0 -> ["Coffee", "Coffee", ...]
    .unique() # makes sure each item is only listed once per price bucket. You don't want a dictionary that says 1.0: "Cookie, Cookie, Cookie..."
    .apply(lambda x: "/".join(map(str, x))) # some prices might map to multiple items, so we join them with a slash. e.g 3.0: "Cake/Juice"
    .to_dict() # convert to dictionary for mapping  
)

print(price_grouping)

{'1.0': 'Cookie', '1.5': 'Tea', '2.0': 'Coffee', '3.0': 'Cake/Juice', '4.0': 'Smoothie/Sandwich', '5.0': 'Salad'}


In [ ]:
# Before using the mapping, ensure the price_per_unit column is numeric float, as it might have been read as object/string type.
import pandas as pd

df['price_per_unit'] = pd.to_numeric(df['price_per_unit'], errors='coerce')

# Price_per_unit is float, but we also need to make sure the keys in the mapping dictionary 'price_grouping' are also float.
# So we convert them.
clean_mapping = {float(k): v for k, v in price_grouping.items()}

# Now we can use this mapping to fill in the missing item values.
df['item'] = df['item'].fillna(df['price_per_unit'].map(clean_mapping))

In [ ]:
# Let's verify our fillna, we have two conditions : item is naN AND price_per_unit is NOT NaN. filter this should give us 0 rows.
# We should still have Nan for item if price_per_unit is also NaN.
missing = df['item'].isna() & df['price_per_unit'].notna()

print(f"Item is Nan X price_per_unit notNan count: {missing.sum()}")

Item is Nan X price_per_unit notNan count: 0


FILLNA FOR QUANTITY, PRICE PER UNIT, TOTAL SPENT

In [38]:
# Next,for 3 columns, quantity, price_per_unit, total_price is connected via a formula:
# total_price = quantity * price_per_unit
# So we can use this formula to impute missing values in any of these 3 columns
# We will create a function on condition that only one of the 3 columns is missing, we can calculate it from the other two.
# If more than one is missing, we cannot impute it.

# Define the columns we are working with
# Purposely using col2 to avoid confusion with col defined earlier
col2 = ['quantity', 'price_per_unit', 'total_spent']

# Convert string columns to float, or else it unsolvable
# errors='coerce' turns non-numeric strings into NaN so they can be solved
for c in col2:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Create a mask for rows where exactly ONE value is missing
mask_one_missing = df[col2].isnull().sum(axis=1) == 1


# Calculate Missing 'total_spent'
df.loc[mask_one_missing & df['total_spent'].isnull(), # filter rows with one missing and total_spent is missing
    'total_spent'] = df['quantity'] * df['price_per_unit'] # then use formula

#  Calculate Missing 'quantity'
df.loc[mask_one_missing & df['quantity'].isnull(), 'quantity'] = \
    df['total_spent'] / df['price_per_unit']

# Calculate Missing 'price_per_unit'
df.loc[mask_one_missing & df['price_per_unit'].isnull(), 'price_per_unit'] = \
    df['total_spent'] / df['quantity']

In [39]:
# Lets verify this
# 'col2' and 'mask_one_missing' are already defined above
# This should produce 0

print(f"Rows with exactly one missing value among {col2}: {mask_one_missing.sum()}")

Rows with exactly one missing value among ['quantity', 'price_per_unit', 'total_spent']: 0
